In [24]:
cd

/Users/emiljaffal


In [46]:
import pandas as pd

file_path = "desktop/data.xlsx"
data = pd.read_excel(file_path)
data

,Formula,Entry,Entry prototype,Structure class,Inv. center,"Unit cell volume, A^3",Density (calc.) [Mg m‰…]
0,CeGa,261037,1,1,1,210.590,6.62
1,CeGa,526690,1,1,1,212.790,6.55
2,CeGa,450659,1,1,1,212.790,6.55
3,CeGa,450525,1,1,1,215.020,6.48
4,CeGa,528215,1,1,1,215.030,6.48
...,...,...,...,...,...,...,...
1447,NiPt,1627941,10,4,1,26.500,15.91
1448,NiPt,455847,10,4,1,26.473,15.92
1449,SnYb,450594,10,4,1,54.147,8.95
1450,SnYb,1823352,10,4,1,54.800,8.84


## How to train chemical formula only datasets

For each chemical formula, we can obtain 438 descriptors consisting of features such as average electronegativity, average boiling points of elements etc. An example of getting descriptors is shown below:


In [52]:
import numpy as np
from jarvis.core.composition import Composition
from jarvis.core.specie import Specie
from jarvis.ai.pkgs.lgbm.regression import regression
from jarvis.ai.descriptors.cfid import get_chem_only_descriptors

formula_unitcell = data[['Formula', 'Unit cell volume, A^3']]

# Convert to NumPy array
X = []
Y = []
IDs = []
for ii, i in enumerate(formula_unitcell):
    X.append(get_chem_only_descriptors(i[0]))
    Y.append(i[1])
    IDs.append(ii)

#Uncomment to convert to array
#X = np.array(X)
#Y = np.array(Y).reshape(-1, 1)
#IDs = np.array(IDs)

Problem here, form 4 total arrays of values, descriptors, values, descriptors

In [53]:
X

[(array([ 4.25150000e+01,  8.50325500e+01,  1.14800000e+01, -7.79840320e+00,
          1.00000000e+00,  1.57560320e+00,  2.00000000e+00,  2.12782116e+02,
          1.26353791e+02, -5.00300000e+01,  2.78500000e-01,  1.30544000e+01,
          5.09300000e+00,  0.00000000e+00,  3.47230000e+00,  1.93249097e+03,
          7.26000000e+00,  9.62300000e+01,  0.00000000e+00,  7.20000000e+00,
          1.34000000e+01,  4.50000000e+00,  0.00000000e+00,  1.50000000e+00,
          0.00000000e+00,  2.90000000e+00,  4.50000000e+00,  1.51000000e+01,
          5.02500000e+01,  3.28255000e+00, -1.41428000e+01, -1.11723000e+01,
         -7.22000000e+00,  1.74202500e+01,  2.09228000e+01,  9.17066546e-01,
         -5.70000000e-02,  2.04700000e+00,  3.24630000e+00,  1.61800000e+00,
          6.23147625e+01,  2.22784032e+01, -3.25230000e+00,  2.63180013e-02,
          1.42857143e-01, -1.00000000e+00,  9.70449960e+00,  1.52942857e+01,
          4.39215686e+03,  2.81760522e+00, -5.35023000e+01,  5.00000000e+00,

Separate into df instead of array

In [66]:
# Create DataFrame
data = {'ID': IDs, 'Unit_Cell_Volume_A3': Y}
df = pd.DataFrame(data)

# Add Chem_Descriptors as columns
for i, chem_desc in enumerate(X):
    df[f'Chem_Descriptors_{i}'] = chem_desc

df

,ID,Unit_Cell_Volume_A3,Chem_Descriptors_0,Chem_Descriptors_1
0,0,o,"[42.515, 85.03255, 11.48, -7.798403199999999, ...","[7350.0, 4200.0914, 0.0, -225.53891, 0.0, 231...."
1,1,n,"[Mean_cfid_bp_mult_atom_rad, Mean_cfid_hfus_ad...","[Mean_cfid_bp_mult_atom_rad, Mean_cfid_hfus_ad..."


Focus on the first column to extract

In [69]:
# Extract Chem_Descriptors_0 from df
chem_desc_0 = df['Chem_Descriptors_0']

# Create a new DataFrame with row 1 and row 2 as columns
new_df = pd.DataFrame({'Row_1': chem_desc_0.iloc[0], 'Row_2': chem_desc_0.iloc[1]})

new_df

,Row_1,Row_2
0,42.515000,Mean_cfid_bp_mult_atom_rad
1,85.032550,Mean_cfid_hfus_add_bp
2,11.480000,Mean_cfid_elec_aff_mult_voro_coord
3,-7.798403,Mean_cfid_mol_vol_subs_atom_mass
4,1.000000,Mean_cfid_is_halogen
...,...,...
1309,15.018403,Min_cfid_atom_mass_subs_X
1310,17.922800,Min_cfid_first_ion_en_add_atom_rad
1311,-3.952300,Min_cfid_therm_cond_subs_X
1312,0.002550,Min_cfid_hfus


Export as csv, import into excel

In [72]:
csv_file_path = 'new_dataframe1.csv'
new_df1.to_csv(csv_file_path, index=False)
print(f"CSV file has been saved to: {csv_file_path}")

CSV file has been saved to: new_dataframe1.csv


Redo here

In [70]:
# Extract Chem_Descriptors_1 from df
chem_desc_1 = df['Chem_Descriptors_1']

# Create a new DataFrame with row 1 and row 2 as columns
new_df1 = pd.DataFrame({'Row_1': chem_desc_1.iloc[0], 'Row_2': chem_desc_1.iloc[1]})

new_df1

,Row_1,Row_2
0,7350.00000,Mean_cfid_bp_mult_atom_rad
1,4200.09140,Mean_cfid_hfus_add_bp
2,0.00000,Mean_cfid_elec_aff_mult_voro_coord
3,-225.53891,Mean_cfid_mol_vol_subs_atom_mass
4,0.00000,Mean_cfid_is_halogen
...,...,...
1309,236.64891,Min_cfid_atom_mass_subs_X
1310,7.94410,Min_cfid_first_ion_en_add_atom_rad
1311,25.62000,Min_cfid_therm_cond_subs_X
1312,0.09140,Min_cfid_hfus


### Names of the 1557 chemical force-field inspired descriptors

### Further breakdown here: 
    https://jarvis-tools.readthedocs.io/en/master/autoapi/jarvis/ai/descriptors/cfid/index.html 

In [50]:
import jarvis
jarvis.ai.descriptors.cfid.feat_names()

['bp_mult_atom_rad',
 'hfus_add_bp',
 'elec_aff_mult_voro_coord',
 'mol_vol_subs_atom_mass',
 'is_halogen',
 'atom_mass_subs_first_ion_en',
 'row',
 'mol_vol_mult_atom_mass',
 'voro_coord_divi_therm_cond',
 'voro_coord_subs_mp',
 'polzbl_mult_atom_rad',
 'elec_aff_mult_X',
 'GV',
 'nfunfill',
 'voro_coord_subs_therm_cond',
 'mp_divi_therm_cond',
 'elec_aff_add_X',
 'mol_vol_add_bp',
 'C-9',
 'C-8',
 'C-7',
 'C-6',
 'C-5',
 'C-4',
 'C-3',
 'C-2',
 'C-1',
 'C-0',
 'mp_subs_elec_aff',
 'hfus_add_elec_aff',
 'elec_aff_subs_first_ion_en',
 'therm_cond_subs_mol_vol',
 'X_subs_mol_vol',
 'first_ion_en_subs_hfus',
 'first_ion_en_add_voro_coord',
 'first_ion_en_divi_atom_mass',
 'atom_rad_subs_polzbl',
 'me1',
 'me3',
 'me2',
 'elec_aff_mult_atom_mass',
 'elec_aff_add_atom_mass',
 'therm_cond_subs_elec_aff',
 'atom_rad_divi_atom_mass',
 'atom_rad_divi_voro_coord',
 'max_oxid_s',
 'polzbl_mult_first_ion_en',
 'mp_divi_voro_coord',
 'mol_vol_divi_hfus',
 'mp_divi_atom_mass',
 'therm_cond_subs_mp'

In [51]:
# Documentation says this should exist...
jarvis.ai.descriptors.elemental.get_element_fraction_desc(formula='SiO2', max_nelements=103)

AttributeError: module 'jarvis.ai.descriptors' has no attribute 'elemental'

## Using with CBFV

***Authors: Steven Kauwe, Andrew Falkowski, Anthony Wang***

Following tutorial here:
    
    https://pypi.org/project/CBFV/

Repo here:

    https://github.com/Kaaiian/CBFV/blob/master/README.md

In [37]:
formula_unitcell

,Formula,"Unit cell volume, A^3"
0,CeGa,210.590
1,CeGa,212.790
2,CeGa,212.790
3,CeGa,215.020
4,CeGa,215.030
...,...,...
1447,NiPt,26.500
1448,NiPt,26.473
1449,SnYb,54.147
1450,SnYb,54.800


In [40]:
import pandas as pd

data = {
    'formula': formula_unitcell['Formula'].tolist(),
    'target': formula_unitcell['Unit cell volume, A^3'].tolist()
}

df = pd.DataFrame(data)
print(df)

     formula   target
0       CeGa  210.590
1       CeGa  212.790
2       CeGa  212.790
3       CeGa  215.020
4       CeGa  215.030
...      ...      ...
1447    NiPt   26.500
1448    NiPt   26.473
1449    SnYb   54.147
1450    SnYb   54.800
1451    SnYb   54.000

[1452 rows x 2 columns]


In [44]:
from CBFV import composition
X, y, formulae, skipped = composition.generate_features(formula_unitcell,
                                                        elem_prop='JARVIS',
                                                        drop_duplicates=False,
                                                        extend_features=True,
                                                        sum_feat=True)

ValueError: list.remove(x): x not in list

## Discrepancy with # of features...

3066 with CBFV vs. 1557 + unknown from element & coloumb matrix features

see documentation: 
    
    https://jarvis-tools.readthedocs.io/en/master/autoapi/jarvis/ai/descriptors/index.html

In [68]:
X #features

,sum_C-0,sum_C-1,sum_C-10,sum_C-11,sum_C-12,sum_C-13,sum_C-14,sum_C-15,sum_C-16,sum_C-17,...,mode_voro_coord_subs_atom_mass,mode_voro_coord_subs_atom_rad,mode_voro_coord_subs_bp,mode_voro_coord_subs_elec_aff,mode_voro_coord_subs_first_ion_en,mode_voro_coord_subs_hfus,mode_voro_coord_subs_mol_vol,mode_voro_coord_subs_mp,mode_voro_coord_subs_polzbl,mode_voro_coord_subs_therm_cond
0,749.1,361.7,0.0,0.0,333.0,333.0,797.5,0.0,0.0,0.0,...,-86.000,8.65,-4526.0,9.494,3.25381,9.7850,1.68,-2418.00,-2.40,-39.0
1,175.8,124.2,0.0,0.0,124.2,124.2,175.8,0.0,0.0,0.0,...,-150.500,10.25,-3188.0,10.816,4.27362,11.8640,-7.01,-1668.00,-12.50,-388.0
2,188.5,176.2,0.0,0.0,106.9,106.9,135.4,0.0,0.0,0.0,...,-102.411,8.45,-1030.0,10.000,1.00618,9.9379,-3.00,-584.22,2.68,-87.0


In [69]:
y #target

0    248.5390
1     66.8444
2     91.5034
Name: target, dtype: float64

In [70]:
formulae #Formulas

0     Tc1V1
1    Cu1Dy1
2     Cd3N2
Name: formula, dtype: object